## Libraries
geopandas may take a while to install. it has to be in python 2.7. ask mateo if there's problem.

In [121]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
from geopandas.tools import overlay
from shapely.geometry.polygon import Polygon
from shapely.geometry.point import Point
import ast
from convertbng.util import convert_bng, convert_lonlat

## Input shapefiles. 
sc_file is source shapefile with attribute values; tg_file is target shapefile (boundary obtained from amenitiy clusterning, in our case)

In [96]:
sc_file="/Users/June/urbanDNA/data/london_info_shp/london_info.shp"
tg_file="/Users/June/urbanDNA/Ele-June/HDBSCAN_polygons.csv"

In [217]:
tg_BNG = pd.read_csv(tg_file)

In [218]:
def project1Polygon (point_list):     
    point_list =ast.literal_eval(point_list)

    for x in range(0,len(point_list)):  
        point_list[x] = point_list[x]

    test_matrix= np.matrix(point_list)
    t= np.matrix.transpose(np.array(convert_bng(test_matrix[:,0],test_matrix[:,1])))

    for x in range(0,len(point_list)):  
        point_list[x]=tuple(t[x])

In [219]:
for i in range(0,len(tg_BNG)):
    project1Polygon(tg_BNG['Polygon'][i])
    tg_BNG.set_value(i,'Polygon',Polygon(ast.literal_eval(tg_BNG['Polygon'][i])))

In [220]:
tg_BNG.head()

,Unnamed: 0,Clusters,Polygon
0,0,0,"POLYGON ((0.139546 51.3517307, 0.1303335 51.35..."
1,1,1,"POLYGON ((-0.0443433 51.4097257, -0.0505941 51..."
2,2,2,"POLYGON ((-0.448596 51.4690074, -0.45114449999..."
3,3,3,"POLYGON ((-0.0933225 51.6409273, -0.0958389999..."
4,4,4,"POLYGON ((-0.2108913 51.3532471, -0.2212 51.35..."


In [80]:
# tg =pd.read_csv(tg_file)

# for i in range(0,len(tg)):
#     tg.set_value(i,'Polygon',Polygon(ast.literal_eval(tg['Polygon'][i]))) ### ast.literal_eval parses list in a string into a string

In [221]:
sc_shp = gpd.read_file(sc_file)
tg_shp = gpd.GeoDataFrame(tg_BNG)
tg_shp.head() #M_W_H_I is medium house income attached from census data.


,Unnamed: 0,Clusters,Polygon
0,0,0,"POLYGON ((0.139546 51.3517307, 0.1303335 51.35..."
1,1,1,"POLYGON ((-0.0443433 51.4097257, -0.0505941 51..."
2,2,2,"POLYGON ((-0.448596 51.4690074, -0.45114449999..."
3,3,3,"POLYGON ((-0.0933225 51.6409273, -0.0958389999..."
4,4,4,"POLYGON ((-0.2108913 51.3532471, -0.2212 51.35..."


In [222]:
tg_shp.columns = ['a', 'Clusters','geometry']
tg_shp.head()

,a,Clusters,geometry
0,0,0,"POLYGON ((0.139546 51.3517307, 0.1303335 51.35..."
1,1,1,"POLYGON ((-0.0443433 51.4097257, -0.0505941 51..."
2,2,2,"POLYGON ((-0.448596 51.4690074, -0.45114449999..."
3,3,3,"POLYGON ((-0.0933225 51.6409273, -0.0958389999..."
4,4,4,"POLYGON ((-0.2108913 51.3532471, -0.2212 51.35..."


In [87]:
sc_shp.head()

,GSS_COD,NAME,HECTARE,NONLD_A,LB_GSS_,BOROUGH,POLY_ID,Ward_nm,pop15,num_jbs,income,accssbl,geometry
0,E05000405,Chessington South,755.173,0.0,E09000021,Kingston upon Thames,50840,Kingston upon Thames Chessington South,10550.0,5300,38310.0,2.4,"POLYGON ((516401.6 160201.8, 516407.3 160210.5..."
1,E05000414,Tolworth and Hook Rise,259.464,0.0,E09000021,Kingston upon Thames,117160,Kingston upon Thames Tolworth and Hook Rise,10650.0,5200,37840.0,2.3,"POLYGON ((517829.6 165447.1, 517837.1 165469.4..."
2,E05000401,Berrylands,145.390,0.0,E09000021,Kingston upon Thames,50449,Kingston upon Thames Berrylands,9800.0,2000,42330.0,2.8,"POLYGON ((518107.5 167303.4, 518114.3 167307.5..."
3,E05000400,Alexandra,268.506,0.0,E09000021,Kingston upon Thames,50456,Kingston upon Thames Alexandra,9700.0,2500,41390.0,2.2,"POLYGON ((520480 166909.8, 520490.7 166901.9, ..."
4,E05000402,Beverley,187.821,0.0,E09000021,Kingston upon Thames,117161,Kingston upon Thames Beverley,10450.0,4500,40700.0,2.8,"POLYGON ((522071 168144.9, 522063.9 168144, 52..."


## Calculation

In [223]:
attr_name ="pop15"

sc_values = pd.to_numeric(sc_shp[attr_name])
tg_zone_value=[]

In [89]:
for i in range(0,len(tg_shp)):
  value=0 # temporary variable whose value gets overwritten with every i. 
  for j in range(0,len(sc_shp)):
    inters=overlay(sc_shp.iloc[[j]],tg_shp.iloc[[i]],how="intersection")
   
    if len(inters) is not 0:
      inters_ratio=inters.geometry.area/tg_shp.geometry.area[i]
      sc_attr_val=sc_values[j] #8 because column 8 is the attribute we need to extract
      inters_wt =inters_ratio*sc_attr_val ## weighted intersection
    else:
       inters_wt=0 #if not intersecting then sc[j]'s contribution is zero
    
    # increment tg_zone_value
    value = value + inters_wt
  
  
  tg_zone_value.append(value)

In [93]:
tg_zone_value #"interpolated value returned. in order. hence can be attached back to dataframe.

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### attach the returned value back to the tg_shp
but test we ggot value for each tg zone first 

In [33]:
len(tg_zone_value) 
len(tg_shp)
## but not really necessary because if the length does not equal,
## can't attach the array to the dataframe anyways.

500

In [34]:
tg_zone_value #is a weird 

type(tg_zone_value[1])
#tg_zone_value


pandas.core.series.Series

In [36]:
tg_shp[attr_name] = tg_zone_value
tg_shp.head()

,OA11CD,POPDEN,AvPTAI2015,PTAL,PTAIHigh,PTAILow,geometry,M_W_H_I
0,E00023264,115.2,14.14550,3,18.49880,11.81340,"POLYGON ((527635.452 171810.058, 527495.957000...",0 737.0 dtype: float64
1,E00003359,36.7,8.62391,2,9.36629,8.01509,"POLYGON ((543581.5159999999 171132.486, 543704...",0 718.0 dtype: float64
2,E00023266,183.4,9.67230,2,11.86470,7.47992,"POLYGON ((526613.539 171839.165, 526546.84 171...",0 737.0 dtype: float64
3,E00020264,96.1,65.49460,6b,67.29880,63.62840,"POLYGON ((533563.5209999999 179695.146, 533351...",0 589.0 dtype: float64
4,E00023263,165.2,14.65720,3,14.65720,14.65720,"POLYGON ((527822 172020, 527979 171903.001, 52...",0 737.0 dtype: float64


# TESTING individual bits of code (Scratch Paper Starts Here)

## Explore geopandas file structure

In [9]:
sc_shp.M_W_H_I.head()

0    None
1     643
2     718
3     638
4     642
Name: M_W_H_I, dtype: object

In [28]:
type(sc_shp.iloc[[2]])

geopandas.geodataframe.GeoDataFrame

#### Calculate area of polygon

geopandaName.geometry.area[row number]

In [11]:
type(sc_shp.geometry.area[2])

numpy.float64

### geo-series?

To access the float value insde the panda.Series

In [142]:
type(tg_zone_value[1])
tg_zone_value[1]
## ok it doesn't really matter because it is still treated as 

0    718.0
dtype: float64

In [138]:
# test_arr=[]
value=0 # temporary variable whose value gets overwritten with every i. 
for j in range(0,len(sc_shp)):
    inters=overlay(sc_shp.iloc[[j]],tg_shp.iloc[[5]],how="intersection")

    if len(inters) is not 0:
      inters_ratio=inters.geometry.area/tg_shp.geometry.area[i]
      sc_attr_val=sc_values[j] 
      inters_wt =inters_ratio*sc_attr_val ## weighted intersection
    else:
       inters_wt=0 #if not intersecting then sc[j]'s contribution is zero

    # increment tg_zone_value
    value += inters_wt 
    value.sum()


AttributeError: 'int' object has no attribute 'sum'

In [136]:
type(value)

pandas.core.series.Series

## Explore "overlap." And plotting. 

#### Intersection

In [112]:
#sc_shp.iloc[[2]]
inter2 = overlay(sc_shp.iloc[[2]] , tg_shp[1:2] , how="intersection")
# if(inter2 is 0):
#     print("null")
len(inter2) ## use len() is 0 or not 0 to check if there is overlap at all.

0    718.0
dtype: float64

In [56]:
# inters = overlay(sc_shp , tg_shp, how="intersection") 
# inters.dtypes

# inters.plot()
# plt.show(1) ## ALWAYS REMEMBER TO SHOW THE PLOT. OTHERWISE IT DOESNT SHOW....!!!!!

## Analogy to regular panda to familiarize with working with dataframe

In [133]:
### analogy to regular panda. basic stuff....

s = pd.DataFrame(np.random.randn(8, 4), columns=['A', 'B', 'C', 'D'])
s.A[2] ### dataframe.columnName[rowNumber]
s.iloc[1] = dict(A=1,B=2,C=1,D=0)
print(s.iloc[1]) # okay printed out vertically

A    1.0
B    2.0
C    1.0
D    0.0
Name: 1, dtype: float64


In [88]:
#s.head(1)
s[0:1] ### the way to slice / return the first row
s.iloc[[0]] ### this is a nicer way..

,A,B,C,D
0,0.214234,0.57994,0.399016,0.09169


In [135]:
pd.to_numeric(s['A'])

0    1.541759
1    1.000000
2   -1.085712
3    1.145229
4    0.573036
5   -0.011416
6   -0.226637
7   -1.364884
Name: A, dtype: float64

## Array (shamelessly...)

In [147]:
t=[]
t.append(1)
t.append(2)
t

[1, 2]

In [21]:
counter = 1

print('This is iteration number: '+ str(counter))

This is iteration number: 1
